# import os
ge

In [7]:
import os, glob, shutil, sys, csv
from pprint import pprint as pp
from Bio.PDB import PDBParser, PDBIO, PPBuilder
from Bio.SeqUtils import seq1
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.Alphabet import generic_protein

In [159]:
# FUNCIONES Y DATOS GLOBALES

path = "/home/emanuel/Documents/trabajo_actual/MegaScript"
verbose = True
data = {}

def cargar_txt_en_lista(file_path):
    return_list = []
    with open(file_path,"r") as openfile:
        for lines in openfile:
            return_list.append(lines.replace("\n",""))
            
    return return_list

def get_uniprot_from_pdb(pdb_id,chain):
    with open("archivos_importantes/pdb_chain_uniprot.csv","r") as openfile:
        temp = csv.reader(openfile,delimiter=",")
        for item in temp:
            if (item[0].upper()== pdb_id) and  (item[1]==chain):
                return item[2]
                break
            else:
                continue
    return None

def get_seq_from_uniprot(uniprot):
    seqfile = "fasta/seq_"+uniprot+".fasta"
    cmd = "wget https://www.uniprot.org/uniprot/"+uniprot+".fasta -O "+seqfile
    os.system(cmd)
    if os.path.isfile:
        for entry in SeqIO.parse(seqfile,"fasta"):
            seq = str(entry.seq)
    return seq

def bajar_estructura(pdb_id):
    os.chdir(path+"/ent_files/")
    filename ="pdb"+pdb_id.lower()+".ent"
    url = "ftp://ftp.wwpdb.org/pub/pdb/data/structures/divided/pdb/"+pdb_id.lower()[1:3]+"/pdb"+pdb_id.lower()+".ent.gz"
    if not os.path.isfile(filename):
        #os.system("wget ftp://ftp.wwpdb.org/pub/pdb/data/structures/all/pdb/pdb"+pdb_id.lower()+".ent.gz")
        wget_code = os.system("wget "+url)
        if wget_code == 2048:
            os.chdir(path)
            return ("OBSOLETE")
        #Uncompress
        os.system("gunzip pdb"+pdb_id.lower()+".ent.gz")
    else:
        if verbose: print (">> File "+filename+" already exists")
    os.chdir(path)
    return filename


def split_pdb_by_chain(pdb_id):
    if not os.path.isdir("pdb_chains/"+pdb_id.upper()):
            os.mkdir("pdb_chains/"+pdb_id.upper())
    actual_pdbfile = PDBParser().get_structure(pdb_id,"ent_files/pdb"+pdb_id.lower()+".ent")
    return_dict = dict()
    for model in actual_pdbfile:
        for chain in model:
            outfilename = pdb_id.upper() + "-" + str(model.get_id()+1) +  "_" + str(chain.get_id()) + ".pdb"
            if not os.path.isfile("pdb_chains/"+pdb_id.upper()+"/"+outfilename):
                io = PDBIO()
                io.set_structure(chain)
                io.save("pdb_chains/"+pdb_id.upper()+"/"+outfilename)
            ppb = PPBuilder().build_peptides(chain)
            this_seq = Seq("",generic_protein)
            for pp in ppb:
                 this_seq += pp.get_sequence()
            return_dict[outfilename]=this_seq
    return return_dict


def read_expdata(path_to_pdbfile):
    with open(path_to_pdbfile,"r") as openfile:
        expdata = ""
        nummodel = 1
        for row in openfile:
            if "EXPDTA" in row:
                expdata +=  (row[9:-1].strip())
                if "X-RAY" in expdata:
                    break
                else:
                    continue
            elif "NUMMDL" in row:
                nummodel = int((row[9:-1].strip()))
                break
            elif "REMARK" in row:
                break
            elif "ATOM" in row:
                break
    return expdata,nummodel



def read_atom_full(pdb_file):
    return_lst = []
    with open(pdb_file,"r") as openfile:
        for row in openfile:
            if row[0:5] == "ATOM ":
                tmp_lst = []
                tmp_lst.append(int(row[6:10+1])) #Integer serial Atom serial number.
                tmp_lst.append(row[12:15+1].strip()) #Atom name Atom name.
                tmp_lst.append(row[16+1]) #Character altLoc Alternate location indicator.
                tmp_lst.append(row[17:19+1]) #Residue name resName Residue name.
                tmp_lst.append(row[21+1]) #Character chainID Chain identifier.
                tmp_lst.append(int(row[22:25+1])) #Integer resSeq Residue sequence number.
                #tmp_lst.append(row[26+1]) #AChar iCode Code for insertion of residues.
                #tmp_lst.append(row[30:37+1]) #Real(8.3) x Orthogonal coordinates for X in Angstroms.
                #tmp_lst.append(row[38:45+1]) #Real(8.3) y Orthogonal coordinates for Y in Angstroms.
                #tmp_lst.append(row[46:53+1]) #Real(8.3) z Orthogonal coordinates for Z in Angstroms.
                #tmp_lst.append(row[54:59+1]) #Real(6.2) occupancy Occupancy.
                #tmp_lst.append(row[60:65+1]) #Real(6.2) tempFactor Temperature factor.
                #tmp_lst.append(row[76:77+1]) #LString(2) element Element symbol, right-justified.
                #tmp_lst.append(row[78:89+1].replace("\n","")) #LString(2) charge Charge on the atom.
                return_lst.append(tmp_lst)
    return return_lst

def print_fixed(query_str,fix=100):
    for index,char in enumerate(query_str):
        print (char,end="")
        if (index+1)%fix ==0:
            print ("")
    print ("")
    return

def save_alidic_to_fasta(ali_dic,outpath):
    with open (outpath,"w") as openfile:
        for itemsin in ali_dic:
            openfile.write (">"+item+"\n")
            openfile.write (ali_dic[item]+"\n")


d = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
     'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N', 
     'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W', 'TER':'*',
     'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M','XAA':'X'}


    


P15452
MKKFLLVAVVGLAGITFANEQLAKQKGCMACHDLKAKKVGPAYADVAKKYAGRKDAVDYLAGKIKKGGSGVWGSVPMPPQNVTDAEAKQLAQWILSIK


In [204]:
ensemble_name = "EJEMPLO1"
ensemble = cargar_txt_en_lista("archivos_importantes/lista_de_conformeros.txt")

uniprot_id = get_uniprot_from_pdb(ensemble[0][0:4],ensemble[0][-1])
uniprot_seq = get_seq_from_uniprot(uniprot_id)

#ensemble_name = "EXAMPLE2"
#ensemble = cargar_txt_en_lista("archivos_importantes/ensemble_P37840.txt")

#toma los primeros 4 elemento chars de cada elemento, arma un conjunto
pdb_to_download = list(set([element[0:4] for element in ensemble ]))
pdb_to_remove = []

#Checkquer estructura de carpetas
folder_list = ["ent_files","fasta","blast","pdb_chains"]
for folder in folder_list:
    if not os.path.isdir(folder):
        print ("Creating folder "+ folder)
        os.mkdir(folder)
    else:
        print (folder+" already exists...-")

#Bajar los enf_files
for pdb_id in pdb_to_download:
    actual_download = bajar_estructura(pdb_id)
    if actual_download == "OBSOLTE":
        pdb_to_remove.append(pdb_id)
        
#punto de control, eliminar cosas relacionadas con pdbs obsoletos.
ensemble = [items for items in ensemble if items[0:4] not in pdb_to_remove]

#le agrega -1 al nombre a aquellas cosas que tienen una unica estructura 
ensemble = [items if "-" in items else (items[0:4]+"-1_"+items[-1]) for items in ensemble ]

#hay que descartar aquellos pdb que fueron marcados como obsoltos
pdb_to_work = list(set(pdb_to_download) - set(pdb_to_remove))

#Separar por cadenas y modelos
pdb_by_chains = dict()
for item in pdb_to_work:
    pdb_by_chains[item] = split_pdb_by_chain(item)
    
#Leer el expdata, cargarla en un diccionario y guardarla en un archivo csv
info_expdta = {}
filename ="archivos_importantes/exp_data_"+ensemble_name+".csv" 
with open(filename,"w") as openfile:
        for item in pdb_to_work:
            temp =read_expdata("ent_files/pdb"+item.lower()+".ent")
            info_expdta[item] = [temp[0],temp[1]]
            openfile.write(item+","+str(temp[0])+","+str(temp[1]))
            openfile.write("\n")  
            

stru_ali = {}
stru_ali[uniprot_id] = uniprot_seq

#carga en memoria la informacion importante de sift (requiere pdb_to_work)
sifts_data = {}
with open("archivos_importantes/pdb_chain_uniprot.csv","r") as openfile:
    temp = csv.reader(openfile,delimiter=",")
    for item in temp:
        if len(item) > 2:
            pdb_id = item[0].upper()
            if pdb_id in pdb_to_work:
                pdb_chain =  pdb_id+"_"+item[1]
                if sifts_data.get(pdb_chain) == None:
                    sifts_data[pdb_chain] = [item[3:]]
                else:
                    sifts_data[pdb_chain].append(item[3:])

#la lista se invierte para que sea mas facil mapear
for keys in sifts_data:
    sifts_data[keys].reverse()
    
#lee la secuencia de cada archivo, 

problematic_items = []
for item in ensemble:
    file_to_read = "pdb_chains/"+item[0:4]+"/"+item + ".pdb"
    seq_vector = ["-"]*len(uniprot_seq)
    for n in read_atom_full(file_to_read):
        if n[1] == "CA":
            pos=  int(n[5])-1
            aa = d[n[3]]
            pdb_chain = item[0:4]+"_"+item[-1]
            
            #print (pdb_chain,pos)
            for sdata in sifts_data[pdb_chain]:
                if pos >= int(sdata[0])-1:
                    val1 = int(sdata[0])-1
                    val2 = int(sdata[4])-1
                    break
                else:
                    continue
            if val2 > val1:
                new_pos = pos - val1 + val2
            elif val2 == val1:
                new_pos = pos - val1
                
            if new_pos < len(uniprot_seq):
                seq_vector[new_pos] = aa
            else:
                print ("ERROR",item,aa,pos,val1,val2,new_pos)
                problematic_items.append(item)
                break
    seq = ""
    for i in seq_vector:
        seq += i
    stru_ali[item] = seq

for n in stru_ali:
    print (n)
    print_fixed (stru_ali.get(n))


#Guarda el alineamiento en el disco
save_alidic_to_fasta(stru_ali,"fasta/"+ensemble_name+"_stru_ali.fasta")
            

ent_files already exists...-
fasta already exists...-
blast already exists...-
pdb_chains already exists...-
>> File pdb1ayg.ent already exists
>> File pdb5aus.ent already exists
>> File pdb5aur.ent already exists
>> File pdb1ynr.ent already exists
>> File pdb4zid.ent already exists
>> File pdb3vym.ent already exists
>> File pdb2ai5.ent already exists


/usr/lib/python3/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 5492.
  PDBConstructionWarning)
/usr/lib/python3/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 5592.
  PDBConstructionWarning)
/usr/lib/python3/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain E is discontinuous at line 5684.
  PDBConstructionWarning)
/usr/lib/python3/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain G is discontinuous at line 5772.
  PDBConstructionWarning)
/usr/lib/python3/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 5858.
  PDBConstructionWarning)
/usr/lib/python3/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 6012.
  PDBConstructionWarning)
/usr/lib/python3/dist-packages/Bio

P15452
MKKFLLVAVVGLAGITFANEQLAKQKGCMACHDLKAKKVGPAYADVAKKYAGRKDAVDYLAGKIKKGGSGVWGSVPMPPQNVTDAEAKQLAQWILSIK
1AYG-10_A
------------------NEQLAKQKGCMACHDLKAKKVGPAYADVAKKYAGRKDAVDYLAGKIKKGGSGVWGSVPMPPQNVTDAEAKQLAQWILSIK
1AYG-11_A
------------------NEQLAKQKGCMACHDLKAKKVGPAYADVAKKYAGRKDAVDYLAGKIKKGGSGVWGSVPMPPQNVTDAEAKQLAQWILSIK
1AYG-12_A
------------------NEQLAKQKGCMACHDLKAKKVGPAYADVAKKYAGRKDAVDYLAGKIKKGGSGVWGSVPMPPQNVTDAEAKQLAQWILSIK
1AYG-13_A
------------------NEQLAKQKGCMACHDLKAKKVGPAYADVAKKYAGRKDAVDYLAGKIKKGGSGVWGSVPMPPQNVTDAEAKQLAQWILSIK
1AYG-14_A
------------------NEQLAKQKGCMACHDLKAKKVGPAYADVAKKYAGRKDAVDYLAGKIKKGGSGVWGSVPMPPQNVTDAEAKQLAQWILSIK
1AYG-15_A
------------------NEQLAKQKGCMACHDLKAKKVGPAYADVAKKYAGRKDAVDYLAGKIKKGGSGVWGSVPMPPQNVTDAEAKQLAQWILSIK
1AYG-16_A
------------------NEQLAKQKGCMACHDLKAKKVGPAYADVAKKYAGRKDAVDYLAGKIKKGGSGVWGSVPMPPQNVTDAEAKQLAQWILSIK
1AYG-17_A
------------------NEQLAKQKGCMACHDLKAKKVGPAYADVAKKYAGRKDAVDYLAGKIKKGGSGVWGSVPMPPQNVTDAEAKQLAQWILSIK
1AYG-18_A
------------

In [211]:
# alieneamiento de homologos y arbol


def run_blast(fasta_file):
    output_filename = "blast_results/"+uniprot_id+"_blast_results.txt"
    cmd = "blastp -query "+ fasta_file +" -db uniprot_sprot.db -evalue 1e-10 -out "+ output_filename +" -outfmt '6 qaccver qlen qstart qend sseqid saccver slen sstart send length staxid nident gaps evalue pident qcovs' -num_alignments 1000"
    os.system(cmd)
    return output_filename




resultados_blast = run_blast("fasta/seq_"+uniprot_id+".fasta")

In [218]:
a =  "qaccver qlen qstart qend sseqid saccver slen sstart send length staxid nident gaps evalue pident qcovs".split(" ")

with open(resultados_blast, "r") as openfile:
    for lines in openfile:
        print (lines.split("\t"))
        break

['qaccver', 'qlen', 'qstart', 'qend', 'sseqid', 'saccver', 'slen', 'sstart', 'send', 'length', 'staxid', 'nident', 'gaps', 'evalue', 'pident', 'qcovs']
['sp|P15452|CY552_HYDTT', '98', '1', '98', 'sp|P15452|CY552_HYDTT', 'sp|P15452|CY552_HYDTT', '98', '1', '98', '98', 'N/A', '98', '0', '2.91e-66', '100.000', '100\n']


In [210]:
def filtrar_output_blast(blast_output):
  # Descartar lo que cumpla con la condicion 35 idetnidad y 70 de coverage
    if verbose: print ("> Applying filters to identity and coverage...")
    output_filename1 = blast_output.replace("blast_output","blast_out_filtered")
    output_filename2 = blast_output.replace("blast_output","blast_out_filtered_300")
    dic_ident = dict()
    for n in range(3,11):
        dic_ident[n] =list()
    lista_iden = list()
    lista_acc = list()
    with open(blast_output,"r") as inputfile:
        for line in inputfile:
            a = line.split()
            identity = float(a[14])
            coverage = float(a[15])
            accession = a[4]
            if (identity > 35) and (identity < 100) and (coverage > 70):
              #deberia ser menor estricto a 100, porque si es 100% identica no estas agarrando la misma?
                with open(output_filename1, 'a') as out:
                    out.write(line)
                coef = int(identity/10)
                lista_iden.append(coef)
                tupla_t = ( accession.split("|")[1],identity)
                dic_ident[coef].append(tupla_t)
                lista_acc.append(accession.split("|")[1])

    borrar=False
    #Esto printea una suerte de histograma de % de identidad.
    if len(lista_iden)< 30:
        warning_message = "WARNING: filtered blast with less that 30 results("+str(len(lista_iden))+" results)"
        prRed (warning_message)
        borrar=True
        #Esta variable indica que no se debe seguir trabajando con estos datos por generar muy pocos resultados de blast.
    if len(lista_iden)> 300:
        warning_message = "WARNING: filtered blast with more that 300 results("+str(len(lista_iden))+" results)"
        prRed (warning_message)

    final_acc_list = select_if_mas_de_300(dic_ident)

    if verbose: print ("> Filtred blast results: "+output_filename1)
    return output_filename1, final_acc_list, borrar

def select_if_mas_de_300 (dic_ident):
    print ("HISTOGRAMA==")
    new_dict = dict()
    for n in range(3,11):
        dic_ident[n].sort() # Ordena el primer el
        if len(dic_ident[n])>20:
            print (n*10,len(dic_ident[n]))
            new_dict[n] = dic_ident[n][0:20]
        else:
            print (n*10,len(dic_ident[n]))
            new_dict[n] = dic_ident[n]
    final_acc_list = list()
    for ranges in new_dict:
        for items in new_dict[ranges]:
            final_acc_list.append(items[0])
    return final_acc_list

In [ ]:
# nota: base de dato de mapeo de residuos pdb/uniprot